In [13]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import pandas as pd
def input():
    
    df = pd.read_csv('QUES.csv')  
    features = ['dit', 'noc', 'mpc', 'rfc','lcom','dac','wmc','nom','size2','size1']
# Separating out the features
    x = df.loc[:, features].values
# Separating out the target
    y = df.loc[:,['change']].values
    

    x = scale(x)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

    data = {'x_train': x_train, 
            'x_test': x_test, 
            'y_train': y_train, 
            'y_test': y_test}

    return data

In [14]:
import numpy as np
input()
from sklearn import metrics
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score
from sklearn.metrics import r2_score


# Helper function that combines the pattern layer and summation layer
def rbf(centre, x, sigma):

    centre = centre.reshape(1, -1)

    temp = -np.sum((centre - x) ** 2, axis = 1)
    temp = temp / (2 * sigma * sigma)
    temp = np.exp(temp)
    gaussian = np.sum(temp)
    
    return gaussian


def subset_by_class(data, labels):

    x_train_subsets = []
    
    for l in labels:
        indices = np.where(data['y_train'] == l)
        x_train_subsets.append(data['x_train'][indices, :])

    return x_train_subsets


def PNN(data):

    num_testset = data['x_test'].shape[0]
    labels = np.unique(data['y_train'])
    num_class = len(labels)

    sigma = 10

    # Splits the training set into subsets where each subset contains data points from a particular class
    x_train_subsets = subset_by_class(data, labels)	

    # Variable for storing the summation layer values from each class
    summation_layer = np.zeros(num_class)
    
    # Variable for storing the predictions for each test data point
    predictions = np.zeros(num_testset)

    for i, test_point in enumerate(data['x_test']):
        
        for j, subset in enumerate(x_train_subsets):
            # Calculate summation layer
            summation_layer[j] = np.sum(
                rbf(test_point, subset[0], sigma)) / subset[0].shape[0] 
        
        # The index having the largest value in the summation_layer is stored as the prediction
        predictions[i] = np.argmax(summation_layer)
    
    return predictions


def print_metrics(y_test, predictions):

    print('Accuracy: {}'.format(accuracy_score(y_test, predictions)))
    print('Precision: {}'.format(precision_score(y_test, predictions, average = 'micro')))
    print('Recall: {}'.format(recall_score(y_test, predictions, average = 'micro')))
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))  
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions))  
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
    print('R Squared Error:', r2_score(y_test, predictions))

if __name__ == '__main__':
    
    data = input()
    predictions = PNN(data)
    print_metrics(data['y_test'], predictions)

Accuracy: 0.0
Precision: 0.0
Recall: 0.0
Mean Absolute Error: 49.59090909090909
Mean Squared Error: 4116.863636363636
Root Mean Squared Error: 64.16279012296485
R Squared Error: -1.0129633838957832
